# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Nguyễn Thị Cẩm Nhung
- MSSV: 21127386
- Lớp: 21CLC08

# Import

In [2998]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2999]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [3000]:
# Cài đặt các hàm cần thiết ở đây

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [3001]:
# Phần code cho yêu cầu 1a
X_train1a = X_train.iloc[:, 0:11]

model1a = LinearRegression()
model1a.fit(X_train1a, y_train)

X_test1a = X_test.iloc[:, 0:11]
y_predict1a = model1a.predict(X_test1a)
print([f'{coef:.3f}' for coef in model1a.coef_], f'{model1a.intercept_:.3f}')

['-23183.330', '702.767', '1259.019', '-99570.608', '18369.962', '1297.532', '-8836.727', '141.760', '145.742', '114.643', '34955.750'] 49248.090


In [3002]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae_1a = mean_absolute_error(y_test, y_predict1a)
print(f'{mae_1a:.3f}')

105052.530


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -23183.330*Gender+ 702.767*10percentage+1259.019*12percentage -99570.608* CollegeTier+ 18369.962*Degree+ 1297.532*collegeGPA-8836.727* CollegeCityTier+ 141.760*English+ 145.742*Logical+ 114.643*Quant+ 34955.750*Domain+ 49248.090$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [3003]:
# Phần code cho yêu cầu 1b
random_indices = np.arange(len(X_train))
np.random.shuffle(random_indices)

X_shuffle = X_train.iloc[random_indices]
y_shuffle = y_train.iloc[random_indices]

features_1b = ['conscientiousness', 'agreeableness', 'extraversion', 'nueroticism', 'openess_to_experience']
y_train1b = y_shuffle.copy()
mae1b_results = []
kf = KFold(n_splits=5)

for feature_idx, feature in enumerate(features_1b):
    mae_feature_results = []
    X_train1b = X_shuffle[feature]
    for train_idx, test_idx in kf.split(X_train1b):
        model1b = LinearRegression()
        model1b.fit(X_train1b.iloc[train_idx].values.reshape(-1, 1), y_train1b.iloc[train_idx])
        y_pred = model1b.predict(X_train1b.iloc[test_idx].values.reshape(-1, 1))

        mae = np.mean(np.abs(y_pred - y_train1b.iloc[test_idx]))
        mae_feature_results.append(mae)

    mean_mae = np.mean(mae_feature_results)
    mae1b_results.append((feature, mean_mae))

# Tìm ra đặc trưng tốt nhất
best_feature_1b, best_mean_mae_1b = min(mae1b_results, key=lambda x: x[1])
print("\nBest feature:", best_feature_1b)
print("Mean MAE for best feature:", best_mean_mae_1b)
# In ra các kết quả cross-validation như yêu cầu
print("Results:")
print("| STT | Mô hình với 1 đặc trưng | MAE  |")
print("|:---:|:-----------------------:|:----:|")
for idx, (feature, mean_mae) in enumerate(mae1b_results):
    print(f"|  {idx+1}  | {feature:24} | {mean_mae:.3f}  |")


Best feature: nueroticism
Mean MAE for best feature: 123421.09291837722
Results:
| STT | Mô hình với 1 đặc trưng | MAE  |
|:---:|:-----------------------:|:----:|
|  1  | conscientiousness        | 124153.947  |
|  2  | agreeableness            | 123680.814  |
|  3  | extraversion             | 123780.149  |
|  4  | nueroticism              | 123421.093  |
|  5  | openess_to_experience    | 123813.700  |


In [3004]:
X_train1b_best_feature = X_train.iloc[:, X_train.columns.get_loc(best_feature_1b):X_train.columns.get_loc(best_feature_1b)+1]
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
model1b = LinearRegression()
model1b.fit(X_train1b_best_feature, y_train)

X_test1b = X_test.iloc[:, X_test.columns.get_loc(best_feature_1b):X_train.columns.get_loc(best_feature_1b)+1]
y_predict1b = model1b.predict(X_test1b)
print([f'{coef:.3f}' for coef in model1b.coef_], f'{model1b.intercept_:.3f}')

['-16021.494'] 304647.553


In [3005]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
mae_1b = mean_absolute_error(y_test, y_predict1b)
print(f'{mae_1b:.3f}')

119361.917


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -16021.494*neuroticism+ 304647.553$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [3006]:
# Phần code cho yêu cầu 1c
random_indices = np.arange(len(X_train))
np.random.shuffle(random_indices)

X_shuffle = X_train.iloc[random_indices]
y_shuffle = y_train.iloc[random_indices]

features_1c = ['English', 'Logical', 'Quant']
y_train1c = y_shuffle.copy()
mae1c_results = []
kf = KFold(n_splits=5)

for feature_idx, feature in enumerate(features_1c):
    mae_feature_results = []
    X_train1c = X_shuffle[feature]
    for train_idx, test_idx in kf.split(X_train1c):
        model1c = LinearRegression()
        model1c.fit(X_train1c.iloc[train_idx].values.reshape(-1, 1), y_train1c.iloc[train_idx])
        y_pred = model1c.predict(X_train1c.iloc[test_idx].values.reshape(-1, 1))

        mae = np.mean(np.abs(y_pred - y_train1c.iloc[test_idx]))
        mae_feature_results.append(mae)

    mean_mae = np.mean(mae_feature_results)
    mae1c_results.append((feature, mean_mae))
# Tìm ra đặc trưng tốt nhất
best_feature_1c, best_mean_mae_1c = min(mae1c_results, key=lambda x: x[1])
print("\nBest feature:", best_feature_1c)
print("Mean MAE for best feature:", best_mean_mae_1c)
# In ra các kết quả cross-validation như yêu cầu
print("Results:")
print("| STT | Mô hình với 1 đặc trưng | MAE  |")
print("|:---:|:-----------------------:|:----:|")
for idx, (feature, mean_mae) in enumerate(mae1c_results):
    print(f"|  {idx+1}  | {feature:24} | {mean_mae:.3f}  |")


Best feature: Quant
Mean MAE for best feature: 117362.23217328615
Results:
| STT | Mô hình với 1 đặc trưng | MAE  |
|:---:|:-----------------------:|:----:|
|  1  | English                  | 120728.913  |
|  2  | Logical                  | 119974.264  |
|  3  | Quant                    | 117362.232  |


In [3007]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
X_train1c_best_feature = X_train.iloc[:, X_train.columns.get_loc(best_feature_1c):X_train.columns.get_loc(best_feature_1c)+1]
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
model1c = LinearRegression()
model1c.fit(X_train1c_best_feature, y_train)

X_test1c = X_test.iloc[:, X_test.columns.get_loc(best_feature_1c):X_train.columns.get_loc(best_feature_1c)+1]
y_predict1c = model1c.predict(X_test1c)
print([f'{coef:.3f}' for coef in model1c.coef_], f'{model1c.intercept_:.3f}')

['368.852'] 117759.729


In [3008]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model
mae_1c = mean_absolute_error(y_test, y_predict1c)
print(f'{mae_1c:.3f}')

108814.060


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 368.852*Quant+ 117759.729$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [3009]:
# Trình bày các phần tìm ra mô hình
# Mô hình 1 là 2 đặc trưng 1b và 1c
random_indices = np.arange(len(X_train))
np.random.shuffle(random_indices)

X_shuffle = X_train.iloc[random_indices]
y_shuffle = y_train.iloc[random_indices]

two_best_feature = [best_feature_1b, best_feature_1c]
y_train1d_1 = y_shuffle.copy()
X_train1d_1 = X_shuffle.loc[:, [best_feature_1b, best_feature_1c]]
# Mô hình 2 toàn bộ đặc trưng
y_train1d_2 = y_shuffle.copy()
X_train1d_2 = X_shuffle
#Mô hình 3 chọn 3 đặc trưng tốt nhất
y_train1d_3 = y_shuffle.copy()
mae1d_3_results = []

for feature_idx, feature in enumerate(X_shuffle.columns):
    mae_feature_results = []
    X_train1d_3 = X_shuffle[feature]
    for train_idx, test_idx in kf.split(X_train1c):
        model1d_3 = LinearRegression()
        model1d_3.fit(X_train1d_3.iloc[train_idx].values.reshape(-1, 1), y_train1d_3.iloc[train_idx])
        y_pred = model1d_3.predict(X_train1d_3.iloc[test_idx].values.reshape(-1, 1))

        mae = np.mean(np.abs(y_pred - y_train1d_3.iloc[test_idx]))
        mae_feature_results.append(mae)

    mean_mae = np.mean(mae_feature_results)
    mae1d_3_results.append((feature, mean_mae))

mae1d_3_results.sort(key=lambda x: x[1])

three_best_features = [feature for feature, _ in mae1d_3_results[:3]]
X_train1d_3 = X_shuffle.copy()
X_train1d_3['New1'] = X_shuffle[three_best_features].sum(axis=1)


print(X_train1d_3.columns)

Index(['Gender', '10percentage', '12percentage', 'CollegeTier', 'Degree',
       'collegeGPA', 'CollegeCityTier', 'English', 'Logical', 'Quant',
       'Domain', 'ComputerProgramming', 'ElectronicsAndSemicon',
       'ComputerScience', 'MechanicalEngg', 'ElectricalEngg', 'TelecomEngg',
       'CivilEngg', 'conscientiousness', 'agreeableness', 'extraversion',
       'nueroticism', 'openess_to_experience', 'New1'],
      dtype='object')


## Thử nghiệm, so sánh các mô hình

In [3010]:
# Phần code cho yêu cầu 1d
#1
mae1d_results = []
mae1d_1_results = []
two_best_features = [best_feature_1b, best_feature_1c]
for train_idx, test_idx in kf.split(X_train1d_1):
    model1d = LinearRegression()
    model1d.fit(X_train1d_1.iloc[train_idx][two_best_features], y_train1d_1.iloc[train_idx])
    y_pred = model1d.predict(X_train1d_1.iloc[test_idx][two_best_features])
    mae = np.mean(np.abs(y_pred - y_train1d_1.iloc[test_idx]))
    mae1d_1_results.append(mae)

mean_mae_1d_1 = np.mean(mae1d_1_results)
mae1d_results.append(("Mô hình 2 features câu 1b và 1c", mean_mae_1d_1))
#2
mae1d_2_results = []
for train_idx, test_idx in kf.split(X_train1d_2):
    model1d = LinearRegression()
    model1d.fit(X_train1d_2.iloc[train_idx], y_train1d_2.iloc[train_idx])
    y_pred = model1d.predict(X_train1d_2.iloc[test_idx])
    mae = np.mean(np.abs(y_pred - y_train1d_2.iloc[test_idx]))
    mae1d_2_results.append(mae)

mean_mae_1d_2 = np.mean(mae1d_2_results)
mae1d_results.append(("Mô hình tất cả features", mean_mae_1d_2))
#3
mae1d_3_results = []
for train_idx, test_idx in kf.split(X_train1d_3):
    model1d = LinearRegression()
    model1d.fit(X_train1d_3.iloc[train_idx], y_train1d_3.iloc[train_idx])
    y_pred = model1d.predict(X_train1d_3.iloc[test_idx])
    mae = np.mean(np.abs(y_pred - y_train1d_3.iloc[test_idx]))
    mae1d_3_results.append(mae)

mean_mae_1d_3 = np.mean(mae1d_3_results)
mae1d_results.append(("Mô hình thêm cột tổng 3 features tốt nhất", mean_mae_1d_3))
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
print("Results:")
print("| STT | Mô hình với 1 đặc trưng | MAE  |")
print("|:---:|:-----------------------:|:----:|")
for idx, (feature, mean_mae) in enumerate(mae1d_results[:3]):
    print(f"|  {idx+1}  | {feature:24} | {mean_mae:.3f}  |")

Results:
| STT | Mô hình với 1 đặc trưng | MAE  |
|:---:|:-----------------------:|:----:|
|  1  | Mô hình 2 features câu 1b và 1c | 117583.751  |
|  2  | Mô hình tất cả features  | 111502.888  |
|  3  | Mô hình thêm cột tổng 3 features tốt nhất | 111446.987  |


In [3011]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
X_train1d_best = X_train.copy()
X_train1d_best["New1"] = X_train[three_best_features].sum(axis=1)
model1d = LinearRegression()
model1d.fit(X_train1d_best, y_train)
X_test1d = X_test.copy()
X_test1d["New1"] = X_test[three_best_features].sum(axis=1)
y_predict1d = model1d.predict(X_test1d)
print([f'{coef:.3f}' for coef in model1d.coef_], f'{model1d.intercept_:.3f}')

['-24977.361', '-2053352396667994.500', '1141.588', '-98567.879', '2147.135', '1352.732', '-6100.622', '140.781', '-2053352396668574.500', '-2053352396668583.500', '28938.320', '76.616', '-50.941', '-180.485', '29.466', '-150.193', '-65.128', '140.289', '-19574.693', '16291.729', '4174.842', '-11231.318', '-5875.195', '2053352396668679.750'] 98510.573


In [3012]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
mae_1d = mean_absolute_error(y_test, y_predict1d)
print(f'{mae_1d:.3f}')

102414.458


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$